In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Import the data set
df = pd.read_csv('Retail_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256170 entries, 0 to 256169
Data columns (total 52 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   _Date                              256170 non-null  object 
 1   Order_ID                           256170 non-null  object 
 2   Invoice                            256170 non-null  object 
 3   Portal                             256170 non-null  object 
 4   Style                              256170 non-null  object 
 5   SKU                                256170 non-null  object 
 6   Qty                                256170 non-null  int64  
 7   Unit_Price                         256170 non-null  float64
 8   Currency                           256170 non-null  object 
 9   conversion_rate                    256170 non-null  float64
 10  Amount                             256170 non-null  float64
 11  Customer_Name                      2561

In [4]:
columns_to_keep = ['Customer ID', 'Portal', 'Unit_Price', 'City', 'Category', 'Size', 'Color', 'SKU', 'Qty']
df_copy = df.loc[:, columns_to_keep]

In [5]:
df_copy.head()

,Customer ID,Portal,Unit_Price,City,Category,Size,Color,SKU,Qty
0,7305,Amazon TBS,424.76,GANDHINAGAR,TUNIC,XXL,White,JNE3686-TU-XXL,1
1,17095,Amazon TBS,424.76,SURAT,TUNIC,XXL,White,JNE3686-TU-XXL,1
2,19481,Amazon TBS,411.43,NEW DELHI,TUNIC,XXL,White,JNE3686-TU-XXL,1
3,34460,Amazon TBS,404.76,CHENNAI,TUNIC,XXL,White,JNE3686-TU-XXL,1
4,56677,Amazon TBS,404.76,NOIDA,TUNIC,XXL,White,JNE3686-TU-XXL,1


In [6]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256170 entries, 0 to 256169
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Customer ID  256170 non-null  int64  
 1   Portal       256170 non-null  object 
 2   Unit_Price   256170 non-null  float64
 3   City         256170 non-null  object 
 4   Category     256170 non-null  object 
 5   Size         256170 non-null  object 
 6   Color        256170 non-null  object 
 7   SKU          256170 non-null  object 
 8   Qty          256170 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 17.6+ MB


In [7]:
# Find number of missing values in each column
df_copy.isna().sum()

Customer ID    0
Portal         0
Unit_Price     0
City           0
Category       0
Size           0
Color          0
SKU            0
Qty            0
dtype: int64

In [8]:
df_copy['Customer ID'].nunique()

100535

In [9]:
# Create CustomerID vs Item (Purchased Items, by StockCode) matrix by pivot table function.
CustomerID_Item_matrix = df_copy.pivot_table(
    index='Customer ID', 
    columns='SKU', 
    values='Qty',
    aggfunc='sum'
)

In [10]:
# Display the shape of matrix, 4372 rows of CustomerID, 3684 columns of Item.
CustomerID_Item_matrix.shape

(100535, 8557)

In [10]:
first_half_matrix = CustomerID_Item_matrix[0:25140]

In [11]:
# Update illustration of the matrix, 1 to represent customer have purchased item, 0 to represent customer haven't purchased.
first_half_matrix = first_half_matrix.applymap(lambda x: 1 if x > 0 else 0)

C:\Users\aggar\AppData\Local\Temp\ipykernel_7504\3614771550.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  first_half_matrix = first_half_matrix.applymap(lambda x: 1 if x > 0 else 0)


In [12]:
# Read header of CustomerID vs Item matrix.
first_half_matrix.loc[12680:].head()

SKU,AN201-RED-L,AN201-RED-M,AN201-RED-S,AN201-RED-XL,AN201-RED-XXL,AN202-ORANGE-L,AN202-ORANGE-M,AN202-ORANGE-S,AN202-ORANGE-XL,AN202-ORANGE-XXL,...,SET445-KR-NP-S,SET445-KR-NP-XXL,SET446-KR-NP-XS,SET449-KR-NP-L,SET449-KR-NP-M,SET449-KR-NP-S,SET449-KR-NP-XL,SET461-KR-NP-L,SET461-KR-NP-M,SET462-KR-NP-S
Customer ID,,,,,,,,,,,,,,,,,,,,,
12680,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12683,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12684,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Create User to User similarity matrix. 
user_to_user_similarity_matrix = pd.DataFrame(
    cosine_similarity(first_half_matrix)
)

In [14]:
# Display header of User to User similarity matrix. 
user_to_user_similarity_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,25130,25131,25132,25133,25134,25135,25136,25137,25138,25139
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Update index to corresponding CustomerID.
user_to_user_similarity_matrix.columns = first_half_matrix.index
user_to_user_similarity_matrix['CustomerID'] = first_half_matrix.index
user_to_user_similarity_matrix = user_to_user_similarity_matrix.set_index('CustomerID')

In [16]:
# Display header of User to User similarity matrix. 
user_to_user_similarity_matrix.head()

Customer ID,0,1,2,3,4,5,6,7,8,9,...,25136,25137,25138,25139,25140,25141,25142,25143,25144,25145
CustomerID,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
# Display CustomerID (12702) purchased items.
target_customer = 12702.0
items_purchased_by_X = set(first_half_matrix.loc[target_customer].iloc[
    first_half_matrix.loc[target_customer].to_numpy().nonzero()].index)
items_purchased_by_X

{'SET366-KR-NP-M'}

In [70]:
# Randomly pick CustomerID (12702) to display the most similar CustomerID.
# The most similar CustomerID is 14608, which has 51% similarity.
similar_customers = user_to_user_similarity_matrix.loc[target_customer].sort_values(ascending=False)
similar_customers

Customer ID
16416    1.000000
8725     1.000000
12702    1.000000
18605    0.707107
13328    0.707107
           ...   
8384     0.000000
8383     0.000000
8382     0.000000
8381     0.000000
25145    0.000000
Name: 12702, Length: 25140, dtype: float64

In [71]:
# Randomly pick CustomerID (12702) to display the most similar CustomerID.
# The most similar CustomerID is 14608, which has 51% similarity.
def top_5_values_not_equal_to_1(similar_customers):
    # Filter values not equal to 1
    filtered_customer = similar_customers[similar_customers != 1]
    
    # Sort the filtered series in descending order
    sorted_customer = filtered_customer.sort_values(ascending=False)
    
    # Take the top 5 values
    top_5_customers = sorted_customer.head(5)
    
    return top_5_customers

# similar_customer = user_to_user_similarity_matrix.loc[4880.0].sort_values(ascending=False)
most_5_similar_customer = top_5_values_not_equal_to_1(similar_customers)
# Access the index values (Customer IDs)
customer_ids = most_5_similar_customer.index

# Convert the index values to a list if needed
customer_ids_list = customer_ids.tolist()
customer_ids_list

[18605, 13328, 5430, 3111, 8391]

In [72]:
# Display CustomerID (14608) purchased items.
# items_purchased_by_Y = set(first_half_matrix.loc[customer_id].iloc[
#     first_half_matrix.loc[customer_id].to_numpy().nonzero()].index)
# items_purchased_by_Y
def get_nonzero_indices(CustomerID):
    # Select the row corresponding to the given CustomerID
    row = first_half_matrix.loc[CustomerID]
    
    # Get the indices where the value is not zero
    nonzero_indices = row.iloc[row.to_numpy().nonzero()].index
    
    # Return the indices as a set
    return set(nonzero_indices)

# Initialize an empty set to store the results
result_set = set()

# Run the code 5 times with different CustomerID values
for i in range(0, len(customer_ids_list)):
    result_set.update(get_nonzero_indices(customer_ids_list[i]))

result_set

{'J0109-SKD-M',
 'J0118-TP-S',
 'JNE3334-KR-XXL',
 'JNE3515-KR-XL',
 'JNE3621-KR-M',
 'JNE3718-KR-M',
 'JNE3797-KR-S',
 'JNE3887-KR-XXL',
 'SET182-KR-DH-XS',
 'SET270-KR-PP-XXL',
 'SET353-KR-NP-L',
 'SET357-KR-NP-XXXL',
 'SET360-KR-NP-M',
 'SET366-KR-NP-M',
 'SET366-KR-NP-S',
 'SET377-KR-NP-XXXL',
 'SET383-KR-NP-XL'}

In [73]:
# Find out items which purchased by X (12702) but not yet purchased by Y (14608).
recommended_items = result_set - items_purchased_by_X

In [74]:
# Display the list of items recommended for Y (14608).
recommended_items

{'J0109-SKD-M',
 'J0118-TP-S',
 'JNE3334-KR-XXL',
 'JNE3515-KR-XL',
 'JNE3621-KR-M',
 'JNE3718-KR-M',
 'JNE3797-KR-S',
 'JNE3887-KR-XXL',
 'SET182-KR-DH-XS',
 'SET270-KR-PP-XXL',
 'SET353-KR-NP-L',
 'SET357-KR-NP-XXXL',
 'SET360-KR-NP-M',
 'SET366-KR-NP-S',
 'SET377-KR-NP-XXXL',
 'SET383-KR-NP-XL'}

In [75]:
# Display the list of items recommended for Y (14608) with item Description.
df.loc[
    df['SKU'].isin(recommended_items), 
    ['SKU', 'Category', 'Color']
].drop_duplicates().set_index('SKU')

,Category,Color
SKU,,
JNE3797-KR-S,KURTA,Green
SET383-KR-NP-XL,SET,Light Green
J0118-TP-S,TOP,Red
JNE3621-KR-M,KURTA,OFF WHITE
JNE3718-KR-M,KURTA,Navy Blue
JNE3887-KR-XXL,KURTA,White
SET182-KR-DH-XS,KURTA SET,Black
JNE3334-KR-XXL,KURTA,Black
SET357-KR-NP-XXXL,SET,Pink
